# 이름/학번

이름: 김수현

학번: 20183911

In [1]:
import numpy as np
import torch

# Example dataset

강의를 위해서 임의의 dataset을 준비하겠습니다.
예제로 봐주시고, 큰 물리적 의미는 부여하지 않겠습니다.

- Data는 장미과와 국화과의 A 효소, B 효소, C 효소, D 효소를 측정한 값이라고 가정합니다.
- Label은 각 sample이 장미인지 (0) 국화인지 (1)에 대한 정보라고 가정합니다.

In [2]:
batch_size = 10 
num_feature = 4
torch.manual_seed(0)

X_batch = torch.randn(batch_size, num_feature)
Y_batch = (torch.sum(X_batch, dim=1)>0).type(torch.float).reshape(-1,1)

In [3]:
X_batch

tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530],
        [-0.2159, -0.7425,  0.5627,  0.2596],
        [-0.1740, -0.6787,  0.9383,  0.4889],
        [ 1.2032,  0.0845, -1.2001, -0.0048],
        [-0.5181, -0.3067, -1.5810,  1.7066]])

In [4]:
Y_batch

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.]])

# Notation 정리

강의자료와 비교하면 

\begin{align*}
\text{X_batch} = 
\begin{bmatrix}
(x^{(1)})^\top\\
(x^{(2)})^\top\\
\vdots \\
(x^{(m)})^\top
\end{bmatrix}, \quad
\text{Y_batch} = 
\begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
\vdots \\
y^{(m)}
\end{bmatrix}
\end{align*}

# Example Problem: Single Neuron

- 한개의 neuron이 있다고 가정하고 $\mathbb{R}^4$ 를 입력받아서 $\mathbb{R}$로 출력한다고 가정합니다.
- Activation 함수는 ReLU 함수, 즉, 
\begin{align*}
\text{ReLU}(x) = \max(0, x)
\end{align*}
를 사용한다고 가정합니다.

Neuron을 통해서 batch sample 전체를
\begin{align*}
Z = \begin{bmatrix}
(w^T x^{(1)} + b)^T \\
(w^T x^{(2)} + b)^T \\
\vdots \\
(w^T x^{(m)} + b)^T
\end{bmatrix} 
\end{align*}
연산을 수행해서 $Z$를 구하세요.

- $w$는 random Gaussian으로 생성하세요. 위에서 예기한 입력과 출력이 맞도록 weight를 생성하세요.
- Bias $b$는 1로 설정




In [5]:
# 답 작성

W = torch.randn(num_feature, 1)
b = 1
Z = torch.matmul(X_batch, W)+b
print(Z)

tensor([[ 2.2910],
        [ 3.0975],
        [ 1.6780],
        [ 1.0120],
        [ 1.4991],
        [-0.5044],
        [ 1.4891],
        [ 1.3363],
        [ 0.2980],
        [-1.8288]])


원하는 연산을 하는지 확인하도록 합니다

In [6]:
z_loop = torch.empty(batch_size, 1)
for i in torch.arange(batch_size):
    z_loop[i,:] = torch.matmul(W.T, X_batch[i,:].T)+b
print(z_loop)

tensor([[ 2.2910],
        [ 3.0975],
        [ 1.6780],
        [ 1.0120],
        [ 1.4991],
        [-0.5044],
        [ 1.4891],
        [ 1.3363],
        [ 0.2980],
        [-1.8288]])


C:\Users\LG\AppData\Local\Temp\ipykernel_15332\1446079948.py:3: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3281.)
  z_loop[i,:] = torch.matmul(W.T, X_batch[i,:].T)+b


## Problem 1
ReLU 함수를 작성하고 위에서 찾은 $Z$의 각 원소에 ReLU 함수를 적용하여 `a`라는 변수에 저장하세요.

- torch.clamp() 함수를 공부하고 적용하세요




In [7]:
# 답 작성:

def ReLU(x):
    return torch.clamp(x, min = 0)
A = ReLU(Z)
print(A)


# torch.clamp(tensor, min= , max=) 형태로 값을 지정할 수 있는데 ReLU 함수의 경우 최대값을 지정할 필요가 없어서 min 값만 지정해줬습니다.

tensor([[2.2910],
        [3.0975],
        [1.6780],
        [1.0120],
        [1.4991],
        [0.0000],
        [1.4891],
        [1.3363],
        [0.2980],
        [0.0000]])


# Single Layer Network

- 한개의 Layer에 $k=5$개의 Neuron 이 있는 network를 구성하고 출력을 구하세요
- Activation function은 모든 neuron에 ReLU를 적용합니다
- 모든 weight는 Gaussian 분포로 랜덤 생성하세요 `torch.randn()`
- $i$ 번째 neuron의 weight들을 $w_i$라고 할때,
\begin{align*}
\text{W} = \begin{bmatrix}
w_1, w_2, w_3, w_4, w_5
\end{bmatrix}
\end{align*}
라고 하고, weight matrix `W`를 만드세요.
  - `W = torch.randn(???, ???)` 으로 생성
- Bias 역시 `b`라는 `tensor`에 저장하고, 각 neuron 별로 `1`로 설정합니다
  - `b = torch.ones(???,???)`
- 아래 problem 2-2에서 수업에서 배운 $Z$ 행렬과 $A$ 행렬을 구하세요

# Problem 2-1
`Z` 행렬과 `A` 행렬의 차원은 어떻게 나와야하나요?

답 작성)

Z 는 (10,5), 
A 는 (10,5)

-> Z행렬에 ReLU를 적용한 것이 A행렬이기 때문에 Z행렬과 A행렬은 차원이 같습니다.

# Problem 2-2 
위에서 요구한 코딩을 완성하세요

In [8]:
# 답 작성
 
W = torch.randn(4,5)
b = torch.ones(1,5)
Z = torch.matmul(X_batch, W)+b
A = ReLU(Z) 

print(A)

# W는 input으로 들어오는 값이 행의 개수, output으로 나가는 값이 열의 개수입니다.
# b는 각 뉴런 별 1로 설정했기 때문에 5개의 뉴런에 1을 더해 줄 수 있도록 길이가 5인 (python)1차원 벡터를 만들어줍니다.
# Z는 X와 W의 선형결합에 b를 더해준 값으로 나타납니다.
# A는 ReLU(Z)입니다.

tensor([[1.5009, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 5.6229, 3.3937, 0.3508, 0.0000],
        [1.9497, 0.0000, 0.4126, 1.3829, 2.4589],
        [1.5663, 3.2170, 0.5114, 0.0000, 0.0751],
        [3.6741, 0.0000, 0.0000, 0.0000, 2.0759],
        [2.1981, 0.0000, 0.7003, 3.8946, 5.3346],
        [2.2619, 0.0000, 0.0000, 0.4367, 1.6558],
        [2.8142, 0.0000, 0.0000, 0.3528, 2.0421],
        [0.0000, 1.7287, 3.6229, 3.6950, 1.9198],
        [0.7818, 0.0000, 1.2625, 3.4941, 3.4477]])


# Problem 3: 

- $x^{(3)}$ 를 입력으로하는 2번째 Neuron의 결과값을 출력하세요
- 위에서 구한 `A[i, j]`  인덱싱을 통해서 출력하세요
- `Python`의 인덱싱은 `0`부터 시작한다는 것을 주의하세요

In [9]:
# 답 작성

print('answer = ', A[2, 1])
# x^(i),i=1,2,...,10 으로 각 x는 위 문제2번 결과의 각 행이고, 5개의 뉴런을 통해 나온 선형연산의 결과는 열로 나타납니다.
# x^(3)을 입력으로 하는 연산 결과는  A[2,:]이고, 3번째 Neuron의 연산 결과는 A[:,1]이므로 x^(3)을 입력으로 하는 2번째 Neuron의 결과값은 A[2,1]입니다.

answer =  tensor(0.)


# Problem 4: Multi-Layer network

- 3개의 layer가 있는 network를 구성합니다.
- 2번째 layer의 입력 크기는 $k^{[1]}=16$, 출력 크기는 $k^{[2]}=6$
- 마지막 layer의 출력은 $k^{[3]}=1$개의 neuron으로 구성
- 각 layer의 연산값을 구하세요. 
  - 각 layer의 선형 변환 결과값은 `Z1`, `Z2`, `Z3`로 저장하세요
  - 각 layer의 결과값은 `A1`, `A2`, `A3`로 저장하세요
- 모든 weight는 Gaussian 랜덤 변수로 생성, bias는 1로 구성된 벡터로 생성함
- 각 layer의 weight는 `W1`, `W2`, `W3`로하고, bias는 `b1`, `b2`, `b3`로 생성함
- Activation 함수는 `ReLU`를 적용하세요
- Loop 없이 행렬연산으로 구생하세요

In [10]:
# 답 작성

W1= torch.randn(4,16)
W2= torch.randn(16,6)
W3= torch.randn(6,1) 
b1= torch.ones(1,16) 
b2= torch.ones(1,6)
b3= 1

Z1 = torch.matmul(X_batch, W1)+b1 
A1 = ReLU(Z1)
Z2 = torch.matmul(A1,W2)+b2
A2 = ReLU(Z2) 
Z3 = torch.matmul(A2,W3)+b3
A3 = ReLU(Z3)


In [11]:
print(Z1.shape)
print(Z1)


torch.Size([10, 16])
tensor([[ 2.5549e+00,  2.2229e+00,  1.9173e+00,  1.0671e+00,  1.8000e+00,
         -8.1495e-01,  4.8913e-01,  2.3724e+00,  6.7394e-01,  7.5445e-01,
         -9.3364e-01,  2.1596e-01, -1.1603e+00, -9.3651e-02,  9.8412e-01,
          1.8653e+00],
        [-1.3552e+00, -1.1635e+00, -2.2510e-02,  1.7841e+00,  1.9706e+00,
          5.6673e+00, -2.8499e+00,  7.0236e-01,  2.9956e+00,  7.3250e-01,
          2.0680e+00,  2.8065e-01,  7.9413e-01, -1.2854e+00,  1.7711e-01,
          3.1582e-01],
        [-7.5565e-01,  2.9468e-01,  2.7113e+00,  7.5281e-01,  2.0534e+00,
         -4.1597e-01,  2.6103e+00,  2.1627e+00,  7.3798e-01, -1.0151e+00,
          2.4346e-02, -8.4654e-01, -3.6184e-01,  7.5211e-01,  2.7299e-02,
          2.3964e+00],
        [ 2.4884e+00,  1.6547e+00,  2.0604e+00, -1.9408e-01,  1.6191e+00,
          2.6641e+00,  6.9943e-01, -2.6340e+00,  1.2683e+00,  1.1058e+00,
          1.8484e+00,  2.7851e+00,  3.0059e+00,  2.7595e+00,  1.2777e-01,
         -2.0744e-01],

In [12]:
print(Z2.shape)
print(Z2)


torch.Size([10, 6])
tensor([[16.9212,  1.0260, -4.6165, -6.3777,  5.9811, -0.3279],
        [ 7.6355,  7.2746,  3.5961, -3.8283,  2.1788,  6.9361],
        [ 9.2321, -1.2725, -7.4480, -7.8591, 11.3802,  0.7822],
        [10.8815,  1.3737,  6.1209,  0.0476, -0.7056, -6.3161],
        [19.8907, -2.5386, -6.3575, -5.6670, 11.8698, -5.2738],
        [12.6349, -6.8004, -4.1671, -8.0727,  8.2638,  0.3457],
        [11.3866, -0.5714, -4.4427, -5.3309,  9.6460, -2.6888],
        [11.4696, -1.1531, -3.9394, -4.8390, 10.6136, -4.9896],
        [12.8070,  3.3260,  4.4189, -6.3869, -0.6832,  6.1099],
        [22.6441, -6.5984,  1.9180, -5.6541, -3.5610, -3.2025]])


In [13]:
print(Z3.shape)
print(Z3)
print(A3)

torch.Size([10, 1])
tensor([[ -3.0082],
        [ -7.8443],
        [ -1.4784],
        [-13.5116],
        [ -3.6889],
        [ -1.9003],
        [ -2.0068],
        [ -2.1303],
        [ -8.6048],
        [ -6.5886]])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


# Problem 5
- 위에서 구한 `A3[i,j]`의 인덱싱을 통해서 $h_\theta(x^{(3)})$ 을 출력하세요
- 역시 `python`인덱싱은 `0` 부터 시작한다는 것을 주의하세요

In [14]:
# 답 작성

print('h_theta(x3) = ', A3[2,:])

h_theta(x3) =  tensor([0.])


# Problem 6
위에서 공부한 것을 함수로 만들어 보도록 하겠습니다.
아래 한 layer의 선형 변환을 연산하는 class를 만들어 보도록 하죠.

- Class는 `my_linear_layer()`
  - `__init__(self, n_input, n_output)` 함수:
    - `self.W` 변수 초기화: Weight 행렬 `self.W`를 램덤 Gaussian 생성 (차원에 맞는...)
    - `self.b` 변수 초기화: bias 벡터 `self.b`를 모두 `1`인 벡터 생성 (차원에 맞는...)
  - `forward(A)` 함수:
    - 입력: `A`는 sample batch $X$ 또는 전 layer에서 들어오는 입력 batch $A^{[\ell-1]}$을 입력하는 자리
    - return 값
      - `Z` 변수는 $A^{[\ell-1]}$의 선형 변환 값, 즉 $Z^{[\ell]}$



In [15]:
# 답 작성
class my_linear_layer():
    def __init__(self, n_input, n_output):
        self.W = torch.randn(n_input, n_output)
        self.b = torch.ones(1, n_output)
    
    def forward(self,A):
        Z = torch.matmul(A,self.W)+self.b
        return Z
    

답을 확인하기 위해서 `n_input=num_feature`과 `n_output = 5` 인 `my_linear_layer` instance 생성

In [21]:
torch.manual_seed(0)

mll = my_linear_layer(num_feature, 5)
mll.forward(X_batch)

tensor([[ 4.0638,  2.2198, -1.7476, -0.3184,  0.2471],
        [ 0.2967, -2.2693, -1.0325, -3.7866,  0.9864],
        [ 2.8471,  1.4401, -0.5475,  1.4418,  1.7400],
        [ 0.7613,  0.5061,  4.5036,  2.8031,  1.9201],
        [ 5.3088,  4.0278, -0.0173,  3.2829,  1.0363],
        [ 0.6880,  2.4766, -0.1730,  3.1860,  1.3804],
        [ 2.9911,  1.8939,  0.7868,  2.1593,  1.5343],
        [ 3.1934,  2.1373,  1.7049,  3.2263,  1.9170],
        [-2.2147, -0.7392, -0.8183, -1.3959,  0.5713],
        [-1.6620,  2.8714,  0.5136,  1.9950, -0.6663]])

In [22]:
print(mll.W)
print(mll.b)

tensor([[-1.1258, -1.1524, -0.2506, -0.4339,  0.5988],
        [-1.5551, -0.3414,  1.8530,  0.4681, -0.1577],
        [ 1.4437,  0.2660,  1.3894,  1.5863,  0.9463],
        [-0.8437,  0.9318,  1.2590,  2.0050,  0.0537]])
tensor([[1., 1., 1., 1., 1.]])


# Building a Linear Layer with `torch.nn`
위에서 수행한 작업을 `pytorch`에서는 `torch.nn.Linear`라는 명령어로 쉽게 구현할 수 있습니다.
아래 예제를 보도록 하죠

In [23]:
import torch.nn as nn
#W = torch.randn(num_feature, 5)
L1 = nn.Linear(num_feature, 5)
Zll = L1(X_batch)
Zll

tensor([[-0.2346,  0.0595,  0.1510, -0.2836, -0.0313],
        [-0.4402,  0.4180, -1.5513, -1.6114,  0.9582],
        [-0.3911, -0.1696,  0.0634, -0.0419,  0.2315],
        [ 0.7794, -0.4919, -0.4290, -0.0730,  0.7808],
        [ 0.0778, -0.3110,  0.9907,  0.6479, -0.2004],
        [-0.3771, -0.8672,  0.2735,  0.4489, -0.4199],
        [ 0.0255, -0.2766,  0.1765,  0.1120,  0.2455],
        [ 0.1800, -0.3671,  0.3306,  0.3356,  0.3165],
        [-0.4906, -0.4770, -1.1830, -0.9128,  0.0698],
        [ 0.1734, -1.2531, -0.0787,  0.2049, -0.9297]],
       grad_fn=<AddmmBackward0>)

In [24]:
L1.weight = nn.Parameter(W.T)
L1.bias.data.fill_(1.0)
Zll2 = L1(X_batch)
Zll2

tensor([[ 1.5009, -0.6663, -0.2224, -0.6280, -0.0453],
        [-1.6313,  5.6229,  3.3937,  0.3508, -2.1027],
        [ 1.9497, -0.3517,  0.4126,  1.3829,  2.4589],
        [ 1.5663,  3.2170,  0.5114, -0.1567,  0.0751],
        [ 3.6741, -3.0020, -2.0296, -0.5144,  2.0759],
        [ 2.1981, -3.0451,  0.7003,  3.8946,  5.3346],
        [ 2.2619, -0.1470, -0.2009,  0.4367,  1.6558],
        [ 2.8142, -0.2460, -0.6185,  0.3528,  2.0421],
        [-1.1247,  1.7287,  3.6229,  3.6950,  1.9198],
        [ 0.7818, -3.0393,  1.2625,  3.4941,  3.4477]],
       grad_fn=<AddmmBackward0>)

In [25]:
Z

tensor([[ 1.5009, -0.6663, -0.2224, -0.6280, -0.0453],
        [-1.6313,  5.6229,  3.3937,  0.3508, -2.1027],
        [ 1.9497, -0.3517,  0.4126,  1.3829,  2.4589],
        [ 1.5663,  3.2170,  0.5114, -0.1567,  0.0751],
        [ 3.6741, -3.0020, -2.0296, -0.5144,  2.0759],
        [ 2.1981, -3.0451,  0.7003,  3.8946,  5.3346],
        [ 2.2619, -0.1470, -0.2009,  0.4367,  1.6558],
        [ 2.8142, -0.2460, -0.6185,  0.3528,  2.0421],
        [-1.1247,  1.7287,  3.6229,  3.6950,  1.9198],
        [ 0.7818, -3.0393,  1.2625,  3.4941,  3.4477]])